In [1]:
import numpy as np
import cv2
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import tensorflow as tf
Adam = tf.keras.optimizers.Adam
ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten , Conv2D, MaxPool2D
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly import subplots
import plotly
from sklearn.model_selection import train_test_split

In [2]:
width = 28
num_classes = 8
trainpath = '/Users/onna/Documents/krotom-train/img-eng/train'
testpath = '/Users/onna/Documents/krotom-train/img-eng/test'
trainImg = [trainpath + "/"+ f for f in listdir(trainpath)]
testImg = [testpath + "/"+ f for f in listdir(testpath)]

In [3]:
def img2data(path):
    rawImgs = []
    labels = []

    for imagePath in (path):
        for item in tqdm(listdir(imagePath)):
            file = join(imagePath, item)
            
            if file[-1] =='g':
                img = cv2.imread(file , cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img ,(width,width))
                rawImgs.append(img)

                name =  imagePath.split('/')
                l = name[len(name) - 1]
            #['Boiled_leaves', 'Green_stalk_GradeA', 'Green_stalk_GradeB', 'Green_stalk_GradeC', 'JUMBO', 'Red_stalk_GradeA', 'Red_stalk_GradeB', 'Red_stalk_GradeC']
            if l == 'Boiled_leaves':
                labels.append([1,0,0,0,0,0,0,0])         
            elif l == 'Green_stalk_GradeA':
                labels.append([0,1,0,0,0,0,0,0])  
            elif l == 'Green_stalk_GradeB':
                labels.append([0,0,1,0,0,0,0,0])
            elif l == 'Green_stalk_GradeC':
                labels.append([0,0,0,1,0,0,0,0])
            elif l == 'JUMBO':
                labels.append([0,0,0,0,1,0,0,0])
            elif l == 'Red_stalk_GradeA':
                labels.append([0,0,0,0,0,1,0,0])
            elif l == 'Red_stalk_GradeB':
                labels.append([0,0,0,0,0,0,1,0])
            elif l == 'Red_stalk_GradeC':
                labels.append([0,0,0,0,0,0,0,1])
    return rawImgs, labels

In [4]:
x_train, y_train = img2data(trainImg)

100%|██████████| 44/44 [00:00<00:00, 228.78it/s]


In [5]:
x_test, y_test = img2data(testImg)

100%|██████████| 10/10 [00:00<00:00, 187.35it/s]


In [6]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [7]:
x_train.shape,y_train.shape,x_test.shape, y_test.shape

((1381, 28, 28), (1381, 8), (352, 28, 28), (352, 8))

In [8]:
train_data = x_train.reshape((x_train.shape[0], 28, 28, 1))
test_data = x_test.reshape((x_test.shape[0], 28, 28, 1))

In [9]:
print(train_data.shape, test_data.shape)

(1381, 28, 28, 1) (352, 28, 28, 1)


In [10]:
NUM_CLASSES = 8
VAL_SIZE = 0.2
RANDOM_STATE = 99
BATCH_SIZE = 256

In [11]:
x_train, x_val, y_train, y_val = train_test_split(train_data, y_train, test_size=VAL_SIZE, random_state=RANDOM_STATE)

x_train.shape, x_val.shape, y_train.shape, y_val.shape

((1104, 28, 28, 1), (277, 28, 28, 1), (1104, 8), (277, 8))

In [12]:
def create_trace(x,y,ylabel,color):
        trace = go.Scatter(
            x = x,y = y,
            name=ylabel,
            marker=dict(color=color),
            mode = "markers+lines",
            text=x
        )
        return trace
    
def plot_accuracy_and_loss(train_model):
    hist = train_model.history
    acc = hist['accuracy']
    val_acc = hist['val_accuracy']
    loss = hist['loss']
    val_loss = hist['val_loss']
    epochs = list(range(1,len(acc)+1))
    
    trace_ta = create_trace(epochs,acc,"Training accuracy", "Green")
    trace_va = create_trace(epochs,val_acc,"Validation accuracy", "Red")
    trace_tl = create_trace(epochs,loss,"Training loss", "Blue")
    trace_vl = create_trace(epochs,val_loss,"Validation loss", "Magenta")
   
    fig = subplots.make_subplots(rows=1,cols=2, subplot_titles=('Training and validation accuracy',
                                                             'Training and validation loss'))
    fig.append_trace(trace_ta,1,1)
    fig.append_trace(trace_va,1,1)
    fig.append_trace(trace_tl,1,2)
    fig.append_trace(trace_vl,1,2)
    fig['layout']['xaxis'].update(title = 'Epoch')
    fig['layout']['xaxis2'].update(title = 'Epoch')
    fig['layout']['yaxis'].update(title = 'Accuracy', range=[0,1])
    fig['layout']['yaxis2'].update(title = 'Loss', range=[0,1])

    plotly.offline.iplot(fig, filename='accuracy-loss')

In [13]:
datagen = ImageDataGenerator(
        rotation_range=0.05,    #Randomly rotate images in the range
        zoom_range=0.2,         #Randomly zoom image
        width_shift_range=0.1,  #Randomly shift images horizontally
        height_shift_range=0.1, #Randomly shift images vertically
        shear_range=0.05        #Randomly shear images
)

datagen.fit(x_train)

In [14]:
# model = Sequential()
model = tf.keras.Sequential()

#1. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), padding = 'Same', input_shape=(28, 28, 1)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Dropout(0.3))

#2. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), padding = 'Same'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))

model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.3))

#3. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Dropout(0.3))

#4. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))

model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.3))


#FULLY CONNECTED LAYER
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Dropout(0.30))

#OUTPUT LAYER
model.add(tf.keras.layers.Dense(8, activation='softmax'))

In [15]:
optimizer = Adam()
model.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics=["accuracy"])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 32)       128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 28, 28, 32)        0         
                                                                 
 dropout (Dropout)           (None, 28, 28, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 28, 28, 32)       128       
 hNormalization)                                        

In [16]:
NO_EPOCHS = 200

history = model.fit(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
                              shuffle=True,
                              epochs=NO_EPOCHS, validation_data = (x_val, y_val),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // BATCH_SIZE)

Epoch 1/200


2022-09-25 17:26:01.106061: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


4/4 [==============================] - 1s 212ms/step - loss: 2.2855 - accuracy: 0.2146 - val_loss: 2.0343 - val_accuracy: 0.2527
Epoch 2/200
4/4 [==============================] - 1s 163ms/step - loss: 1.7937 - accuracy: 0.4057 - val_loss: 1.9318 - val_accuracy: 0.3141
Epoch 3/200
4/4 [==============================] - 1s 165ms/step - loss: 1.6848 - accuracy: 0.4611 - val_loss: 1.8865 - val_accuracy: 0.3141
Epoch 4/200
4/4 [==============================] - 1s 203ms/step - loss: 1.5888 - accuracy: 0.4873 - val_loss: 1.9908 - val_accuracy: 0.3141
Epoch 5/200
4/4 [==============================] - 1s 207ms/step - loss: 1.5982 - accuracy: 0.4811 - val_loss: 2.2655 - val_accuracy: 0.3141
Epoch 6/200
4/4 [==============================] - 1s 172ms/step - loss: 1.5854 - accuracy: 0.4953 - val_loss: 2.6465 - val_accuracy: 0.3141
Epoch 7/200
4/4 [==============================] - 1s 233ms/step - loss: 1.5438 - accuracy: 0.4917 - val_loss: 3.0291 - val_accuracy: 0.3141
Epoch 8/200
4/4 [========

In [17]:
plot_accuracy_and_loss(history)

In [23]:
score = model.evaluate(test_data, y_test,verbose=0)
print("Test Loss:",score[0])
print("Test Accuracy:",score[1])

Test Loss: 1.8765311241149902
Test Accuracy: 0.4837545156478882
